In [1]:
df = sqlContext.sql("select * from Restaurant_Reviews")
df = df.withColumnRenamed('Liked','class').withColumnRenamed('Review','text')
df.printSchema()

In [2]:
from pyspark.sql.functions import length
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
df = df.withColumn('length',length(df['text']))
df.groupby('class').mean().show()

In [4]:
df = df.withColumn('label', df['class'].cast('float'))
df.printSchema()

In [5]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
df_clean = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [6]:
data_prep_pipe = Pipeline(stages=[tokenizer,stopremove,count_vec,idf,df_clean])
cleaner = data_prep_pipe.fit(df)
clean_data = cleaner.transform(df)
clean_data = clean_data.select(['label','features'])

In [7]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [8]:
nb = NaiveBayes()
liked_predictor = nb.fit(training)

In [9]:
test_results = liked_predictor.transform(testing)
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))